In [5]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Charger le fichier CSV dans un DataFrame
data = pd.read_csv('data.csv')

# Sort les colonnes titres, descriptions, genres
sorted_data = data[['title', 'overview', 'genres']]

# Filtrage des données (impossible de deviner un genre si description < 10 caractères)
filtered_data = sorted_data[sorted_data['overview'].str.len() > 10]

# Tokenisation et lemmatisation avec NLTK
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return lemmatized

# Appliquer la pré-traitement à la colonne 'overview'
filtered_data['processed_overview'] = filtered_data['overview'].apply(preprocess_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                            Description  \
3827  [craig, and, smokey, are, two, guy, in, los, a...   
491   [set, in, 79, a.d., ,, pompeii, tell, the, epi...   
3748  [two, woman, ,, nic, and, jules, ,, brought, a...   
3742  [there, 's, a, deadly, zombie, epidemic, threa...   
287   [with, the, help, of, a, german, bounty, hunte...   
...                                                 ...   
4798  [el, mariachi, just, want, to, play, his, guit...   
4007  [100, pound, overweight, ,, loaded, up, on, st...   
4579  [king, arthur, ,, accompanied, by, his, squire...   
1770  [financial, tv, host, lee, gate, and, his, pro...   
1469  [a, store, clerk, and, an, ice, cream, truck, ...   

                                         Generated Type  
3827                     [{"id": 35, "name": "Comedy"}]  
491                       [{"id": 18, "name": "Drama"}]  
3748                      [{"id": 18, "name": "Drama"}]  
3742  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...  
2